# Generate thermospheric density globe data:

This notebook generates the data used in the other tutorial: `Plotting Globe Data`. Its purpose is to show how to generate ML-based thermospheric density maps across the entire globe

## Imports

In [9]:
import karman
import os
import pandas as pd
import pickle
import numpy as np

## Strategy:

We take the data at a given time, and we generate the globe ata by only varying latitude, longitude, and local solar time.

For instance, we first load the scalers:

In [3]:
base_path='/'+os.getcwd().strip('docs/source/tutorials')
with open(os.path.join(base_path,'tests/data/thermo_scaler.pickle'), 'rb') as handle:
    thermo_scaler = pickle.load(handle)

with open(os.path.join(base_path,'tests/data/cyclical_feature_scaler.pickle'), 'rb') as handle:
    cyclical_feature_scaler = pickle.load(handle)

with open(os.path.join(base_path,'tests/data/thermospheric_density_scaler.pickle'), 'rb') as handle:
    thermospheric_density_scaler = pickle.load(handle)

In [ ]:
'/Users/ga00693/karman/karman/tests/data/'

We then create the dataset:

In [14]:
#we store the features to exclude (not used in training):
#we exclude Ap because we trained an ML model using only JB-08 inputs
features_to_exclude_thermo=['all__dates_datetime__',
                            'tudelft_thermo__ground_truth_thermospheric_density__[kg/m**3]',
                            'tudelft_thermo__satellite__',
                            'celestrack__ap_h_0__',
                            'celestrack__ap_h_1__',
                            'celestrack__ap_h_2__',
                            'celestrack__ap_h_3__',
                            'celestrack__ap_h_4__',
                            'celestrack__ap_h_5__',
                            'celestrack__ap_h_6__',
                            'celestrack__ap_average__',
                            'JB08__thermospheric_density__[kg/m**3]',
                            'NRLMSISE00__thermospheric_density__[kg/m**3]']

dataset=karman.ThermosphericDensityDataset(
    directory=os.path.join(base_path,'tests/data/'),
    features_to_exclude_thermo=features_to_exclude_thermo,
    create_cyclical_features=True,
    max_altitude=600000,
    thermo_scaler=thermo_scaler,
    cyclical_feature_scaler=cyclical_feature_scaler,
    thermospheric_density_scaler=thermospheric_density_scaler
)

Creating thermospheric density dataset
Creating cyclical features
['all__dates_datetime__', 'tudelft_thermo__ground_truth_thermospheric_density__[kg/m**3]', 'tudelft_thermo__satellite__', 'celestrack__ap_h_0__', 'celestrack__ap_h_1__', 'celestrack__ap_h_2__', 'celestrack__ap_h_3__', 'celestrack__ap_h_4__', 'celestrack__ap_h_5__', 'celestrack__ap_h_6__', 'celestrack__ap_average__', 'JB08__thermospheric_density__[kg/m**3]', 'NRLMSISE00__thermospheric_density__[kg/m**3]']
['all__day_of_year__[d]', 'all__seconds_in_day__[s]', 'all__sun_right_ascension__[rad]', 'all__sun_declination__[rad]', 'all__sidereal_time__[rad]', 'tudelft_thermo__longitude__[deg]', 'tudelft_thermo__local_solar_time__[h]']
Used features: Index(['all__year__[y]', 'tudelft_thermo__altitude__[m]',
       'tudelft_thermo__latitude__[deg]',
       'space_environment_technologies__f107_average__',
       'space_environment_technologies__f107_obs__',
       'space_environment_technologies__s107_obs__',
       'space_environm

We then create a dictionary using the first line of the data (i.e., fixed time), and varying only longitude and latitude (it is also important to update local solar time, as that is dependent on longitude):

In [11]:
data=dataset.data_thermo['data']
num_points=40
longitude=np.linspace(-180,180,num_points)
latitude=np.linspace(-90,90,num_points)
lon_grid, lat_grid = np.meshgrid(longitude, latitude)

dic={}
line0=data.iloc[0]
for feature in list(line0.keys()):
    dic[feature]=[]
solar_longitude=line0['tudelft_thermo__longitude__[deg]']+180-(line0['tudelft_thermo__local_solar_time__[h]']-12)*15#lst=(longitude_satellite – longitude_sun)/15 + 12
for i in range(len(longitude)):
    for j in range(len(longitude)):
        for feature in list(line0.keys()):
            if feature not in ['tudelft_thermo__latitude__[deg]', 'tudelft_thermo__longitude__[deg]','tudelft_thermo__local_solar_time__[h]']:
                dic[feature].append(line0[feature])
        dic['tudelft_thermo__latitude__[deg]'].append(lat_grid[i,j])
        dic['tudelft_thermo__longitude__[deg]'].append(lon_grid[i,j])
        dic['tudelft_thermo__local_solar_time__[h]'].append((lon_grid[i,j]-solar_longitude+180)/15+12)

We can finally transform the dictionary into a pandas dataframe, and save it to file to be able to use it (see: `plot_globe_data.ipynb` for an example on how to use it to predict thermospheric density with Karman-ML models).

In [13]:
#we can then transform it as a pandas dataframe
df=pd.DataFrame(dic)
df.head()
#and eventually save it
#df.to_hdf('jb08_nrlmsise_all.h5')

,all__day_of_year__[d],all__year__[y],all__seconds_in_day__[s],tudelft_thermo__altitude__[m],tudelft_thermo__longitude__[deg],tudelft_thermo__latitude__[deg],tudelft_thermo__local_solar_time__[h],space_environment_technologies__f107_average__,space_environment_technologies__f107_obs__,celestrack__ap_average__,...,all__sun_right_ascension__[rad]_sin,all__sun_right_ascension__[rad]_cos,all__sun_declination__[rad]_sin,all__sun_declination__[rad]_cos,all__sidereal_time__[rad]_sin,all__sidereal_time__[rad]_cos,tudelft_thermo__longitude__[deg]_sin,tudelft_thermo__longitude__[deg]_cos,tudelft_thermo__local_solar_time__[h]_sin,tudelft_thermo__local_solar_time__[h]_cos
0,61.0,2004.0,17.0,499077.46875,-180.000000,-90.0,-11.995278,108.199997,110.0,21.0,...,-0.306088,0.952003,-0.131547,0.99131,-0.980069,-0.19866,-0.986758,0.162201,0.647013,-0.762479
1,61.0,2004.0,17.0,499077.46875,-170.769231,-90.0,-11.379893,108.199997,110.0,21.0,...,-0.306088,0.952003,-0.131547,0.99131,-0.980069,-0.19866,-0.986758,0.162201,0.647013,-0.762479
2,61.0,2004.0,17.0,499077.46875,-161.538462,-90.0,-10.764508,108.199997,110.0,21.0,...,-0.306088,0.952003,-0.131547,0.99131,-0.980069,-0.19866,-0.986758,0.162201,0.647013,-0.762479
3,61.0,2004.0,17.0,499077.46875,-152.307692,-90.0,-10.149124,108.199997,110.0,21.0,...,-0.306088,0.952003,-0.131547,0.99131,-0.980069,-0.19866,-0.986758,0.162201,0.647013,-0.762479
4,61.0,2004.0,17.0,499077.46875,-143.076923,-90.0,-9.533739,108.199997,110.0,21.0,...,-0.306088,0.952003,-0.131547,0.99131,-0.980069,-0.19866,-0.986758,0.162201,0.647013,-0.762479
